# Loop 4 Analysis: C++ Overlap Detection Mismatch

## Problem Identified
The jonathanchan C++ SA optimizer improved score from 70.68 to 51.66 but introduced overlaps in 176/200 N configurations. When validated with Python/Shapely, these overlaps were detected and repaired, returning the score to 70.68.

## Root Cause
The C++ overlap detection algorithm differs from Python/Shapely's algorithm. The C++ code uses:
1. Point-in-polygon test (ray casting)
2. Segment intersection test

But Shapely uses a more robust algorithm that handles edge cases differently.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
import os
import glob

getcontext().prec = 30
scale_factor = Decimal('1e15')

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h
        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x * scale_factor), yoff=float(self.center_y * scale_factor))

def load_configuration_from_df(n, df):
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"])[1:] if str(row["x"]).startswith('s') else str(row["x"])
        y = str(row["y"])[1:] if str(row["y"]).startswith('s') else str(row["y"])
        deg = str(row["deg"])[1:] if str(row["deg"]).startswith('s') else str(row["deg"])
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

def get_tree_list_side_length(tree_list):
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(max(bounds[2] - bounds[0], bounds[3] - bounds[1])) / scale_factor

def get_score(trees, n):
    if not trees:
        return 0.0
    side = get_tree_list_side_length(trees)
    return float(side ** 2 / Decimal(n))

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def score_submission(file_path, max_n=200):
    df = pd.read_csv(file_path)
    total_score = 0.0
    failed_overlap_n = []
    for n in range(1, max_n + 1):
        trees = load_configuration_from_df(n, df)
        if trees:
            current_score = get_score(trees, n)
            total_score += current_score
            if has_overlap(trees):
                failed_overlap_n.append(n)
    return total_score, failed_overlap_n

print("Functions loaded")

Functions loaded


In [2]:
# Check available pre-optimized CSVs
preopt_dir = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized'
print("Available pre-optimized CSVs:")
for f in sorted(os.listdir(preopt_dir)):
    if f.endswith('.csv'):
        path = os.path.join(preopt_dir, f)
        try:
            score, overlaps = score_submission(path)
            print(f"  {f}: score={score:.6f}, overlaps={len(overlaps)}")
        except Exception as e:
            print(f"  {f}: ERROR - {e}")

Available pre-optimized CSVs:


  best_ensemble.csv: score=70.676102, overlaps=12


  ensemble.csv: score=70.676102, overlaps=13


  santa-2025.csv: score=70.676102, overlaps=12


  submission.csv: score=70.676501, overlaps=11


In [3]:
# Check the chistyakov folder
chistyakov_dir = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/chistyakov'
if os.path.exists(chistyakov_dir):
    print("\nChistyakov CSVs:")
    for f in sorted(os.listdir(chistyakov_dir)):
        if f.endswith('.csv'):
            path = os.path.join(chistyakov_dir, f)
            try:
                score, overlaps = score_submission(path)
                print(f"  {f}: score={score:.6f}, overlaps={len(overlaps)}")
            except Exception as e:
                print(f"  {f}: ERROR - {e}")


Chistyakov CSVs:


  submission_best.csv: score=70.926150, overlaps=0


In [4]:
# Check telegram folder
telegram_dir = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram'
if os.path.exists(telegram_dir):
    print("\nTelegram CSVs:")
    for f in sorted(os.listdir(telegram_dir)):
        if f.endswith('.csv'):
            path = os.path.join(telegram_dir, f)
            try:
                score, overlaps = score_submission(path)
                print(f"  {f}: score={score:.6f}, overlaps={len(overlaps)}")
            except Exception as e:
                print(f"  {f}: ERROR - {e}")


Telegram CSVs:


  71.97.csv: score=71.972027, overlaps=0


  72.49.csv: score=72.495739, overlaps=0


In [5]:
# Check our current best submission
print("\nCurrent submission:")
score, overlaps = score_submission('/home/submission/submission.csv')
print(f"  Score: {score:.6f}")
print(f"  Overlaps: {overlaps if overlaps else 'None'}")
print(f"  Gap to target (68.92): {score - 68.922808:.6f} points")


Current submission:


  Score: 70.682741
  Overlaps: None
  Gap to target (68.92): 1.759933 points


## Key Insight: The Problem is Overlap Detection Mismatch

The C++ optimizer is producing configurations that it thinks are valid (no overlaps), but Python/Shapely detects overlaps. This means:

1. **The optimizer IS working** - it found better configurations (51.66 vs 70.68)
2. **The overlap detection is the bottleneck** - C++ and Python disagree on what's an overlap
3. **We need a PURE PYTHON optimizer** that uses Shapely for overlap detection

## Options:

### Option A: Pure Python Backpacking (crodoc kernel)
- Start from N=200, work backward to N=1
- For each N, try removing each tree from N+1 configuration
- Uses Shapely for overlap detection (guaranteed correct)
- Slower but produces valid results

### Option B: Fix C++ Overlap Detection
- Add buffer/tolerance to C++ overlap detection
- Make it stricter than Shapely to ensure no false negatives
- Requires modifying C++ code and recompiling

### Option C: Hybrid Approach
- Run C++ optimizer to generate candidates
- Validate each candidate with Python/Shapely
- Only keep configurations that pass Python validation
- This is what we tried, but 176/200 failed

### Option D: Python SA Optimizer with Shapely
- Implement SA optimizer in pure Python
- Use Shapely for overlap detection
- Slower but guaranteed correct

In [6]:
# Let's analyze which N values have the most room for improvement
# by comparing our baseline to the theoretical minimum

df_baseline = pd.read_csv('/home/code/experiments/003_preoptimized/repaired_baseline.csv')

print("Score breakdown by N range:")
ranges = [(1, 50), (51, 100), (101, 150), (151, 200)]
for start, end in ranges:
    range_score = 0
    for n in range(start, end + 1):
        trees = load_configuration_from_df(n, df_baseline)
        if trees:
            range_score += get_score(trees, n)
    print(f"  N={start}-{end}: {range_score:.4f}")

print(f"\nTotal: {sum([get_score(load_configuration_from_df(n, df_baseline), n) for n in range(1, 201)]):.6f}")

Score breakdown by N range:


  N=1-50: 19.0424


  N=51-100: 17.6414


  N=101-150: 17.1495


  N=151-200: 16.8495



Total: 70.682741


In [7]:
# Find the N values with the worst efficiency (highest score contribution per tree)
print("\nWorst-performing N values (highest score per N):")
scores_by_n = []
for n in range(1, 201):
    trees = load_configuration_from_df(n, df_baseline)
    if trees:
        score = get_score(trees, n)
        scores_by_n.append((n, score))

# Sort by score (descending)
scores_by_n.sort(key=lambda x: x[1], reverse=True)

print("Top 20 worst N values:")
for n, score in scores_by_n[:20]:
    print(f"  N={n}: {score:.6f}")

print(f"\nSum of worst 20: {sum([s for _, s in scores_by_n[:20]]):.4f}")
print(f"If improved by 10%: saves {sum([s for _, s in scores_by_n[:20]]) * 0.1:.4f} points")


Worst-performing N values (highest score per N):


Top 20 worst N values:
  N=1: 0.661250
  N=2: 0.450779
  N=3: 0.434745
  N=5: 0.416850
  N=4: 0.416545
  N=7: 0.399897
  N=6: 0.399610
  N=9: 0.387415
  N=8: 0.385407
  N=15: 0.379203
  N=10: 0.376630
  N=21: 0.376451
  N=20: 0.376057
  N=11: 0.375736
  N=22: 0.375258
  N=16: 0.374128
  N=26: 0.373997
  N=12: 0.372724
  N=13: 0.372323
  N=25: 0.372144

Sum of worst 20: 8.0771
If improved by 10%: saves 0.8077 points


## Key Insight: smartmanoj tree_packer_v21 + backward propagation

The smartmanoj kernel has a sophisticated C++ optimizer with:
1. **tree_packer_v21**: Multi-start SA with swap moves, squeeze, compaction, local search
2. **bp.cpp**: Backward propagation that removes boundary-touching trees

The key difference from jonathanchan is that smartmanoj runs backward propagation AFTER the SA optimizer, which can help propagate good configurations from larger N to smaller N.

Let me check if this approach might work better.

In [8]:
# Let's look at what the best_ensemble.csv has that our repaired baseline doesn't
# The best_ensemble has 12 overlaps but scores 70.676 vs our 70.683

df_best_ensemble = pd.read_csv('/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/best_ensemble.csv')
df_repaired = pd.read_csv('/home/code/experiments/003_preoptimized/repaired_baseline.csv')

print("Comparing best_ensemble.csv vs repaired_baseline.csv:")
print("\nN values where best_ensemble is better (ignoring overlaps):")

better_n = []
for n in range(1, 201):
    trees_ensemble = load_configuration_from_df(n, df_best_ensemble)
    trees_repaired = load_configuration_from_df(n, df_repaired)
    
    if trees_ensemble and trees_repaired:
        score_ensemble = get_score(trees_ensemble, n)
        score_repaired = get_score(trees_repaired, n)
        
        if score_ensemble < score_repaired - 1e-9:
            has_ovl = has_overlap(trees_ensemble)
            improvement = score_repaired - score_ensemble
            better_n.append((n, improvement, has_ovl))

print(f"Found {len(better_n)} N values where best_ensemble is better:")
for n, improvement, has_ovl in sorted(better_n, key=lambda x: -x[1])[:20]:
    ovl_str = " (HAS OVERLAP)" if has_ovl else ""
    print(f"  N={n}: improvement={improvement:.6f}{ovl_str}")

# Count how many have overlaps
with_overlap = sum(1 for _, _, has_ovl in better_n if has_ovl)
without_overlap = len(better_n) - with_overlap
print(f"\nWith overlap: {with_overlap}, Without overlap: {without_overlap}")

Comparing best_ensemble.csv vs repaired_baseline.csv:

N values where best_ensemble is better (ignoring overlaps):


Found 12 N values where best_ensemble is better:
  N=134: improvement=0.003286 (HAS OVERLAP)
  N=137: improvement=0.002019 (HAS OVERLAP)
  N=170: improvement=0.000672 (HAS OVERLAP)
  N=61: improvement=0.000217 (HAS OVERLAP)
  N=41: improvement=0.000193 (HAS OVERLAP)
  N=171: improvement=0.000143 (HAS OVERLAP)
  N=77: improvement=0.000060 (HAS OVERLAP)
  N=114: improvement=0.000018 (HAS OVERLAP)
  N=110: improvement=0.000018 (HAS OVERLAP)
  N=117: improvement=0.000008 (HAS OVERLAP)
  N=43: improvement=0.000002 (HAS OVERLAP)
  N=30: improvement=0.000002 (HAS OVERLAP)

With overlap: 12, Without overlap: 0
